In [31]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import xgboost as xgb
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import StandardScaler
import random
import tensorflow.keras.utils as utils
import pydot
from tensorflow.keras import regularizers
from tensorflow.keras import layers
from sklearn.ensemble import RandomForestRegressor
class Mics_Model:
    def __init__(self, dataset_dir, use_encoder=True, sampling_method="Vanilla", global_model="NN", group_number = 2, company_number = 1):
        self.dataset_dir = dataset_dir
        self.use_encoder = use_encoder
        self.sampling_method = sampling_method
        self.global_model = global_model
        self.group_number = group_number
        self.company_number = company_number
        self.raw_data = None
            
    #ASSUMPTION: column 0: index, column 1: labels, remaining columns are features. 
    def get_raw_data(self, index_col=0):
        raw_data = pd.read_csv(self.dataset_dir, index_col=0)
        raw_data = raw_data.fillna(raw_data.mean())
        raw_data = raw_data.sample(frac=1, random_state=41)
        self.raw_data = raw_data
        
    #This method assigns the feature number = column number - 1 (exclude label column). After that, it returns a list of
    #input feature numbers according to group count. Ex: for 28 cols, 27 features, 4 group_num: returns [7,7,6,7] 
    #Output of this function can be fed to get_model methods as inp_sizes input.
    def get_input_group_lenthgs(self):
        count = self.group_number
        input_sizes = [None]*count
        feature_num = len(self.raw_data.columns) - 1
        for i in range(count):
            group_size = round(feature_num/(count-i))
            input_sizes[i] = group_size
            feature_num = feature_num - group_size
        return input_sizes
    
    #This method returns grouped column numbers
    #[[1,4,5],[2,3,6]]
    def get_grouped_feature_cols(self):
        grouped_feature_cols = [None]*self.group_number
        feature_num = len(self.raw_data.columns) - 1
        inp_sizes = self.get_input_group_lenthgs()
        total_nums = [i for i in range(feature_num)]
        for j in range(len(inp_sizes)):
            size = inp_sizes[j]
            temp_list = random.sample(total_nums, size)
            grouped_feature_cols[j] = temp_list
            for k in temp_list:
                total_nums.remove(k)
        return grouped_feature_cols
    
    #groups is a list of lists [[1,4,5], [2,3,6]] which is output of get_grouped_feature_cols method
    #returns: [[train_x1, train_x2..., train_xn, train_y],
    #          [test_x1, test_x2..., test_xn, test_y]]
    def get_features_and_labels(self, groups, random_seed=41):
        row_num = len(self.raw_data.index)
        
        trainx_df = self.raw_data.iloc[:int(0.8*row_num), 1:]
        trainy_df = self.raw_data.iloc[:int(0.8*row_num), 0]
        testx_df = self.raw_data.iloc[int(0.8*row_num):, 1:]
        testy_df = self.raw_data.iloc[int(0.8*row_num):, 0]        
        
        scaler = StandardScaler()
        #trainx_scaled = pd.DataFrame(scaler.fit_transform(trainx_df), columns = trainx_df.columns, index = trainx_df.index)
        #textx_scaled = pd.DataFrame(scaler.transform(testx_df), columns = testx_df.columns, index = testx_df.index)
        
        features_and_labels = [[None for _ in range(self.group_number + 1)] for _ in range(2)]
        
        for index, group in enumerate(groups):
            train_temp = trainx_df.iloc[:,group]
            train_temp_companies = self.transform_dataset(df=train_temp, random_seed=random_seed)
            features_and_labels[0][index] = train_temp_companies.values
            test_temp = testx_df.iloc[:,group]
            test_temp_companies = self.transform_dataset(df=test_temp, random_seed=random_seed)            
            features_and_labels[1][index] = test_temp_companies.values
        #trainy_df_companies = trainy_df.sample(frac=1, random_state=random_seed)
        #testy_df_companies = testy_df.sample(frac=1, random_state=random_seed)        
        features_and_labels[0][self.group_number] = trainy_df.values
        features_and_labels[1][self.group_number] = testy_df.values   
        return features_and_labels
    #returns [[train_x1, train_x2..., train_xn, train_y],
    #         [test_x1, test_x2..., test_xn, test_y]]
    
    
    #For a 10k row telecommunication data, it splits data into #group_num row groups and transforms them independently, adds one-hot encoding.
    #There is no label column here, all columns are features.
    def transform_dataset(self, df, random_seed=41):
        group_num = self.company_number
        col_num = len(df.columns)
        row_num = len(df.index)
        dfs = [None]*group_num
        dfs_features = [None]*group_num
        dfs_scaleds = [None]*group_num
        dfs_new = [None]*group_num
        scaler = StandardScaler()
        for i in range(group_num):
            dfs[i] = df.iloc[int(i/group_num*row_num):int((i+1)/group_num*row_num), :]
            dfs[i] = dfs[i].sample(frac=1, axis=1)
            df_features_scaled_temp = pd.DataFrame(scaler.fit_transform(dfs[i]), columns = dfs[i].columns, index = dfs[i].index)
            
            #dfs_new[i] = df_features_scaled_temp
            dfs_new[i] = dfs[i]
            
            dfs_new[i]['group'] = i
            cols_num = len(dfs_new[i].columns)
            col_names = [j for j in range(cols_num)]
            dfs_new[i].columns = col_names
        df_final = pd.concat(dfs_new, axis=0)
        last_col_num = cols_num - 1
        df_new = df_final.rename(columns={last_col_num: 'group'})
        df_final_onehot = pd.concat([df_new.iloc[:,:-1], pd.get_dummies(df_new.group, prefix='group')], axis=1)
        #df_final_onehot = df_final_onehot.sample(frac=1, random_state=random_seed)
        return df_final_onehot
    
    def get_vanilla_encoder_model(self, inp_size):
        inputs = keras.layers.Input(shape=(inp_size+self.company_number))
        h1 = keras.layers.Dense(10, activation="relu")(inputs)
        h1 = keras.layers.Dense(10, activation="relu")(inputs)        
        outputs = keras.layers.Dense(inp_size, activation="relu")(h1)
        return keras.Model(inputs,outputs)
    
    #This subclass is created for sampling for a given mean and log_variance.
    class Sampling(layers.Layer):
        def call(self, inputs):
            z_mean, z_log_var = inputs
            batch = tf.shape(z_mean)[0]
            dim = tf.shape(z_mean)[1]
            epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
            return z_mean + tf.exp(0.5 * z_log_var) * epsilon #multiplies with std
    
    def get_variatonal_encoder_model(self, inp_size):
        inputs = layers.Input(shape=(inp_size+self.company_number))
        h1 = layers.Dense(10, activation="relu")(inputs)
        z_mean = layers.Dense(inp_size, name="z_mean")(h1)
        z_log_var = layers.Dense(inp_size, name="z_log_var")(h1)
        outputs = self.Sampling()([z_mean, z_log_var])
        return keras.Model(inputs,outputs)
    #New sampling methods can be added here 
    
    def get_nn_model(self, inp_sizes, drop_out=0.25, hidden_num = 4, hidden_size=32, activation="relu"):
        inp_group_count = len(inp_sizes)
        inputs = [None]*inp_group_count
        for i in range(inp_group_count):
            inputs[i] = keras.layers.Input(shape=(inp_sizes[i]+self.company_number), name="input_"+str(i))
        if self.use_encoder == True:
            encoders = [None]*inp_group_count
            if self.sampling_method == "Vanilla":
                for j in range(inp_group_count):
                    encoders[j] = self.get_vanilla_encoder_model(inp_sizes[j])
            elif self.sampling_method == "Variational":
                for j in range(inp_group_count):
                    encoders[j] = self.get_variatonal_encoder_model(inp_sizes[j])
            #This place can be extended if new sampling methods are added.
            global_inputs = [None]*inp_group_count
            for k in range(inp_group_count):
                global_inputs[k] = encoders[k](inputs[k])
            global_input = keras.layers.concatenate(global_inputs)
        else:
            global_input = keras.layers.concatenate(inputs)
            
        h = keras.layers.Dense(hidden_size, activation="relu", kernel_regularizer=regularizers.l1_l2(l1=1e-4, l2=1e-3))(global_input)
        h = keras.layers.Dropout(drop_out)(h)
        for hidden in range(hidden_num):
            h = keras.layers.Dense(hidden_size, activation="relu", kernel_regularizer=regularizers.l1_l2(l1=1e-4, l2=1e-3))(h)
            h = keras.layers.Dropout(drop_out)(h) 

        outputs = keras.layers.Dense(1, activation=activation)(h)    
        return keras.Model(inputs=inputs, outputs = outputs) 
    
    def default_exp(self, batch_size = 300):
        inp_sizes = self.get_input_group_lenthgs()
        groups = self.get_grouped_feature_cols()
        features_and_labels = self.get_features_and_labels(groups)
        MICS_model = self.get_nn_model(inp_sizes=inp_sizes)
        callback = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=50), 
                keras.callbacks.ReduceLROnPlateau("val_loss", factor = 0.8, patience=30,
                                                 verbose = 2, mode = "auto", 
                                                  min_lr = 1e-6)]
        MICS_model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.01), loss=keras.losses.MeanSquaredError())
        history = MICS_model.fit(x = features_and_labels[0][:-1], y = features_and_labels[0][-1],  
                                 validation_data = (features_and_labels[1][:-1], features_and_labels[1][-1]),
                                 epochs=300, batch_size = batch_size, callbacks=callback)
        training_val_loss = history.history["val_loss"]
        best_row_index = np.argmin(training_val_loss)
        best_val_loss = training_val_loss[best_row_index]
        print(best_val_loss)
        
    def default_exp_house(self, batch_size = 300):
        inp_sizes = self.get_input_group_lenthgs()
        groups = self.get_grouped_feature_cols()
        features_and_labels = self.get_features_and_labels(groups)
        MICS_model = self.get_nn_model(inp_sizes=inp_sizes, activation="sigmoid")
        callback = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=50), 
                keras.callbacks.ReduceLROnPlateau("val_loss", factor = 0.8, patience=30,
                                                 verbose = 2, mode = "auto", 
                                                  min_lr = 1e-6)]
        MICS_model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.01), loss=keras.losses.BinaryCrossentropy(), metrics=["accuracy"])
        history = MICS_model.fit(x = features_and_labels[0][:-1], y = features_and_labels[0][-1],  
                                 validation_data = (features_and_labels[1][:-1], features_and_labels[1][-1]),
                                 epochs=300, batch_size = batch_size, callbacks=callback)
        training_val_loss = history.history["val_loss"]
        best_row_index = np.argmin(training_val_loss)
        best_val_loss = training_val_loss[best_row_index]
        
        training_acc = history.history["val_accuracy"]
        best_row_index_acc = np.argmax(training_acc)
        best_val_acc = training_acc[best_row_index_acc]        
        print("best val loss is: " + str(best_val_loss))
        print("best val accuracy is: " + str(best_val_acc))
        return best_val_acc
                


In [32]:
dataset_dir = "./Datasets/Energy_Small.csv"
deneyelim = Mics_Model(dataset_dir, use_encoder=True, group_number=2, company_number=1)
#deneyelim.get_raw_data()
#deneyelim.raw_data
#deneyelim.default_exp()

In [33]:
deneyelim.get_raw_data()
deneyelim.raw_data

,Appliances,lights,T1,RH_1,T2,RH_2,T3
5,50,40,19.890000,46.026667,19.20,44.500000,19.79
6,60,50,19.890000,45.766667,19.20,44.500000,19.79
7,60,50,19.856667,45.560000,19.20,44.500000,19.73
8,60,40,19.790000,45.597500,19.20,44.433333,19.73
1,60,30,19.890000,46.693333,19.20,44.722500,19.79
9,70,40,19.856667,46.090000,19.23,44.400000,19.79
2,50,30,19.890000,46.300000,19.20,44.626667,19.79
4,60,40,19.890000,46.333333,19.20,44.530000,19.79
3,50,40,19.890000,46.066667,19.20,44.590000,19.79
0,60,30,19.890000,47.596667,19.20,44.790000,19.79


In [34]:
inp_sizes = deneyelim.get_input_group_lenthgs()
inp_sizes

[3, 3]

In [35]:
groups = deneyelim.get_grouped_feature_cols()
groups

[[5, 2, 0], [3, 1, 4]]

In [36]:
features_and_labels = deneyelim.get_features_and_labels(groups)
features_and_labels

[[array([[40.        , 46.02666667, 19.79      ,  1.        ],
         [50.        , 45.76666667, 19.79      ,  1.        ],
         [50.        , 45.56      , 19.73      ,  1.        ],
         [40.        , 45.5975    , 19.73      ,  1.        ],
         [30.        , 46.69333333, 19.79      ,  1.        ],
         [40.        , 46.09      , 19.79      ,  1.        ],
         [30.        , 46.3       , 19.79      ,  1.        ],
         [40.        , 46.33333333, 19.79      ,  1.        ]]),
  array([[19.2       , 19.89      , 44.5       ,  1.        ],
         [19.2       , 19.89      , 44.5       ,  1.        ],
         [19.2       , 19.85666667, 44.5       ,  1.        ],
         [19.2       , 19.79      , 44.43333333,  1.        ],
         [19.2       , 19.89      , 44.7225    ,  1.        ],
         [19.23      , 19.85666667, 44.4       ,  1.        ],
         [19.2       , 19.89      , 44.62666667,  1.        ],
         [19.2       , 19.89      , 44.53      ,  1. 

In [23]:
features_and_labels[0][2]

array([110,  60,  30, ..., 440, 360,  50])

In [26]:
dataset_dir = "./Datasets/energydata_use.csv"
dff = pd.read_csv(dataset_dir)
dff = dff.iloc[:10, 1:8]
dff

,Appliances,lights,T1,RH_1,T2,RH_2,T3
0,60,30,19.890000,47.596667,19.20,44.790000,19.79
1,60,30,19.890000,46.693333,19.20,44.722500,19.79
2,50,30,19.890000,46.300000,19.20,44.626667,19.79
3,50,40,19.890000,46.066667,19.20,44.590000,19.79
4,60,40,19.890000,46.333333,19.20,44.530000,19.79
5,50,40,19.890000,46.026667,19.20,44.500000,19.79
6,60,50,19.890000,45.766667,19.20,44.500000,19.79
7,60,50,19.856667,45.560000,19.20,44.500000,19.73
8,60,40,19.790000,45.597500,19.20,44.433333,19.73
9,70,40,19.856667,46.090000,19.23,44.400000,19.79


In [28]:
dff.to_csv("./Datasets/Energy_Small.csv")

In [37]:
dff

,Appliances,lights,T1,RH_1,T2,RH_2,T3
0,60,30,19.890000,47.596667,19.20,44.790000,19.79
1,60,30,19.890000,46.693333,19.20,44.722500,19.79
2,50,30,19.890000,46.300000,19.20,44.626667,19.79
3,50,40,19.890000,46.066667,19.20,44.590000,19.79
4,60,40,19.890000,46.333333,19.20,44.530000,19.79
5,50,40,19.890000,46.026667,19.20,44.500000,19.79
6,60,50,19.890000,45.766667,19.20,44.500000,19.79
7,60,50,19.856667,45.560000,19.20,44.500000,19.73
8,60,40,19.790000,45.597500,19.20,44.433333,19.73
9,70,40,19.856667,46.090000,19.23,44.400000,19.79


In [38]:
group = [1,3,5]
train_temp = dff.iloc[:,group]
train_temp_companies = train_temp.sample(frac=1, random_state=41)